# Algoritmo de Asignación de Tareas

In [15]:
import numpy as np
import random
from scipy.optimize import linear_sum_assignment

In [16]:
def generar_costos_aleatorios(num_trabajadores, num_tareas):
    costos = np.random.randint(1, 101, size=(num_trabajadores, num_tareas))
    print("\nMatriz de costos generada aleatoriamente:")
    print(costos)
    return costos

In [17]:
def rellenar_matriz(costos):
    filas, columnas = costos.shape
    if filas == columnas:
        return costos
    if filas > columnas:
        padding = np.full((filas, filas - columnas), 9999)
        costos = np.hstack((costos, padding))
    else:
        padding = np.full((columnas - filas, columnas), 9999)
        costos = np.vstack((costos, padding))
    return costos

In [18]:
def algoritmo_asignacion(costos):
    fila_indices, columna_indices = linear_sum_assignment(costos)
    asignaciones = list(zip(fila_indices, columna_indices))
    costo_total = costos[fila_indices, columna_indices].sum()
    return asignaciones, costo_total

In [19]:
print("Algoritmo de asignación de tareas\n")
num_trabajadores = int(input("Número de trabajadores: "))
num_tareas = int(input("Número de tareas: "))

costos = generar_costos_aleatorios(num_trabajadores, num_tareas)
matriz_completa = rellenar_matriz(costos)

asignaciones, costo_total = algoritmo_asignacion(matriz_completa)

print("\nAsignaciones óptimas:")
for trabajador, tarea in asignaciones:
    if trabajador < num_trabajadores and tarea < num_tareas:
        print(f"Trabajador {trabajador + 1} → Tarea {tarea + 1} (Costo: {costos[trabajador][tarea]})")
print(f"\nCosto total mínimo: {costo_total}")

Algoritmo de asignación de tareas


Matriz de costos generada aleatoriamente:
[[51 67  9 43 61]
 [94 36 29 12 87]
 [92 73 56 69 87]
 [35 65 36 27 86]
 [ 7 49 34 68 35]]

Asignaciones óptimas:
Trabajador 1 → Tarea 3 (Costo: 9)
Trabajador 2 → Tarea 4 (Costo: 12)
Trabajador 3 → Tarea 2 (Costo: 73)
Trabajador 4 → Tarea 1 (Costo: 35)
Trabajador 5 → Tarea 5 (Costo: 35)

Costo total mínimo: 164
